In [1]:
import os
import pathlib
import sys

default_dir_path = str(pathlib.Path(os.getcwd()).parent.absolute())

sys.path.append(default_dir_path)

from properties import ApplicationProperties

[2020-12-11 12:21:20,406][INFO][properties.py:22] Complete to apply the random seed, RANDOM_SEED : 777


In [2]:
from torch import optim
from torch.nn import functional as F
import torch

from dataset.factory import DatasetFactory
from domain.hyperparameters import Hyperparameters
from logger import logger
from model.factory import ModelFactory
from trainer.factory import TrainerFactory

In [3]:
from dataset.nsl_kdd_dataset import NSLKDDDataFrameFactory, NSLKDDDataset

In [4]:
import numpy as np
import pandas as pd

In [5]:
entire_df = NSLKDDDataFrameFactory.create("entire")

[2020-12-11 12:21:21,324][INFO][nsl_kdd_dataset.py:98] Success to load dataset dataframe : train
[2020-12-11 12:21:21,387][INFO][nsl_kdd_dataset.py:98] Success to load dataset dataframe : test
[2020-12-11 12:21:22,690][INFO][nsl_kdd_dataset.py:110] Success to load dataset dataframe : entire


In [29]:
entire_df.describe()

,index,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,148517.000000,148517.000000,1.485170e+05,1.485170e+05,148517.000000,148517.000000,148517.000000,148517.000000,148517.000000,148517.000000,...,148517.000000,148517.000000,148517.000000,148517.000000,148517.000000,148517.000000,148517.000000,148517.000000,148517.000000,148517.000000
mean,55136.045530,276.779305,4.022795e+04,1.708885e+04,0.000215,0.020523,0.000202,0.189379,0.004323,0.402789,...,183.928042,119.462661,0.534521,0.084103,0.145932,0.030584,0.256122,0.251304,0.136220,0.136397
std,38372.764374,2460.683131,5.409612e+06,3.703525e+06,0.014677,0.240069,0.019417,2.013160,0.072248,0.490461,...,98.528328,111.232318,0.448061,0.194102,0.308638,0.108975,0.428500,0.429719,0.322741,0.335282
min,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,18564.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,87.000000,11.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,51714.000000,0.000000,4.400000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,72.000000,0.600000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,88843.000000,0.000000,2.780000e+02,5.710000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,255.000000,255.000000,1.000000,0.070000,0.050000,0.010000,0.600000,0.500000,0.000000,0.000000
max,125972.000000,57715.000000,1.379964e+09,1.309937e+09,1.000000,3.000000,3.000000,101.000000,5.000000,1.000000,...,255.000000,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [32]:
entire_df[entire_df["label"] == "normal"]

,index,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,0,tcp,ftp_data,SF,491,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,1,0,udp,other,SF,146,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
3,3,0,tcp,http,SF,232,8153,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,4,0,tcp,http,SF,199,420,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
12,12,0,tcp,http,SF,287,2251,0,0,0,...,219,1.00,0.00,0.12,0.03,0.00,0.00,0.00,0.00,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148506,22533,0,tcp,http,SF,274,1623,0,0,0,...,255,1.00,0.00,0.01,0.04,0.00,0.00,0.00,0.00,normal
148508,22535,0,tcp,http,SF,280,6087,0,0,0,...,255,1.00,0.00,0.20,0.04,0.00,0.00,0.00,0.00,normal
148512,22539,0,tcp,smtp,SF,794,333,0,0,0,...,141,0.72,0.06,0.01,0.01,0.01,0.00,0.00,0.00,normal
148513,22540,0,tcp,http,SF,317,938,0,0,0,...,255,1.00,0.00,0.01,0.01,0.01,0.00,0.00,0.00,normal


In [6]:
x_train, y_train, x_test, y_test = NSLKDDDataFrameFactory.preprocess(
    dataframe=entire_df,
    abnormal_sample_ratio=0.1,
    normal_abnormal_ratio=1.0,
    val_test_ratio=0.3,
    is_shuffle=True
)

[2020-12-11 12:21:23,675][INFO][nsl_kdd_dataset.py:156] Entire data size : 148517
[2020-12-11 12:21:23,677][INFO][nsl_kdd_dataset.py:157] Normal data size : 77054
[2020-12-11 12:21:23,678][INFO][nsl_kdd_dataset.py:158] Abnormal data size : 7146
[2020-12-11 12:21:23,678][INFO][nsl_kdd_dataset.py:159] Normal test data size : 7146
[2020-12-11 12:21:23,679][INFO][nsl_kdd_dataset.py:160] Training data size : 69908
[2020-12-11 12:21:23,680][INFO][nsl_kdd_dataset.py:161] Test data size : 14292


In [7]:
x_train

,index,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
121730,121730,0,318,501,0,0,0,0,0,1,...,28,255,1.00,0.00,0.04,0.02,0.0,0.0,0.0,0.0
115367,115367,0,292,1091,0,0,0,0,0,1,...,12,255,1.00,0.00,0.08,0.03,0.0,0.0,0.0,0.0
72379,72379,0,319,5036,0,0,0,0,0,1,...,5,255,1.00,0.00,0.20,0.03,0.0,0.0,0.0,0.0
512,512,0,356,1125,0,0,0,0,0,1,...,36,255,1.00,0.00,0.03,0.03,0.0,0.0,0.0,0.0
6921,6921,0,250,608,0,0,0,0,0,1,...,255,255,1.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32624,32624,17044,146,105,0,0,0,0,0,0,...,255,1,0.00,0.56,0.89,0.00,0.0,0.0,0.0,0.0
147057,21084,0,1,0,0,0,0,0,0,0,...,255,93,0.36,0.45,1.00,0.00,0.0,0.0,0.0,0.0
102734,102734,7,122,11930,0,0,0,0,0,1,...,189,3,0.02,0.03,0.01,0.00,0.0,0.0,0.0,0.0
97559,97559,0,43,45,0,0,0,0,0,0,...,255,166,0.65,0.01,0.16,0.00,0.0,0.0,0.0,0.0


In [8]:
x_train

,index,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
121730,121730,0,318,501,0,0,0,0,0,1,...,28,255,1.00,0.00,0.04,0.02,0.0,0.0,0.0,0.0
115367,115367,0,292,1091,0,0,0,0,0,1,...,12,255,1.00,0.00,0.08,0.03,0.0,0.0,0.0,0.0
72379,72379,0,319,5036,0,0,0,0,0,1,...,5,255,1.00,0.00,0.20,0.03,0.0,0.0,0.0,0.0
512,512,0,356,1125,0,0,0,0,0,1,...,36,255,1.00,0.00,0.03,0.03,0.0,0.0,0.0,0.0
6921,6921,0,250,608,0,0,0,0,0,1,...,255,255,1.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32624,32624,17044,146,105,0,0,0,0,0,0,...,255,1,0.00,0.56,0.89,0.00,0.0,0.0,0.0,0.0
147057,21084,0,1,0,0,0,0,0,0,0,...,255,93,0.36,0.45,1.00,0.00,0.0,0.0,0.0,0.0
102734,102734,7,122,11930,0,0,0,0,0,1,...,189,3,0.02,0.03,0.01,0.00,0.0,0.0,0.0,0.0
97559,97559,0,43,45,0,0,0,0,0,0,...,255,166,0.65,0.01,0.16,0.00,0.0,0.0,0.0,0.0


In [9]:
x_train

,index,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
121730,121730,0,318,501,0,0,0,0,0,1,...,28,255,1.00,0.00,0.04,0.02,0.0,0.0,0.0,0.0
115367,115367,0,292,1091,0,0,0,0,0,1,...,12,255,1.00,0.00,0.08,0.03,0.0,0.0,0.0,0.0
72379,72379,0,319,5036,0,0,0,0,0,1,...,5,255,1.00,0.00,0.20,0.03,0.0,0.0,0.0,0.0
512,512,0,356,1125,0,0,0,0,0,1,...,36,255,1.00,0.00,0.03,0.03,0.0,0.0,0.0,0.0
6921,6921,0,250,608,0,0,0,0,0,1,...,255,255,1.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32624,32624,17044,146,105,0,0,0,0,0,0,...,255,1,0.00,0.56,0.89,0.00,0.0,0.0,0.0,0.0
147057,21084,0,1,0,0,0,0,0,0,0,...,255,93,0.36,0.45,1.00,0.00,0.0,0.0,0.0,0.0
102734,102734,7,122,11930,0,0,0,0,0,1,...,189,3,0.02,0.03,0.01,0.00,0.0,0.0,0.0,0.0
97559,97559,0,43,45,0,0,0,0,0,0,...,255,166,0.65,0.01,0.16,0.00,0.0,0.0,0.0,0.0


In [10]:
train_df = NSLKDDDataFrameFactory.create("train")
test_df = NSLKDDDataFrameFactory.create("test")

[2020-12-11 12:21:24,572][INFO][nsl_kdd_dataset.py:98] Success to load dataset dataframe : train
[2020-12-11 12:21:24,644][INFO][nsl_kdd_dataset.py:98] Success to load dataset dataframe : test


In [11]:
train_df

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,tcp,private,S0,0,0,0,0,0,0,...,25,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune
125969,8,udp,private,SF,105,145,0,0,0,0,...,244,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,normal
125970,0,tcp,smtp,SF,2231,384,0,0,0,0,...,30,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,normal
125971,0,tcp,klogin,S0,0,0,0,0,0,0,...,8,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune


In [12]:
test_df

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,private,REJ,0,0,0,0,0,0,...,10,0.04,0.06,0.00,0.00,0.00,0.0,1.00,1.00,neptune
1,0,tcp,private,REJ,0,0,0,0,0,0,...,1,0.00,0.06,0.00,0.00,0.00,0.0,1.00,1.00,neptune
2,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,86,0.61,0.04,0.61,0.02,0.00,0.0,0.00,0.00,normal
3,0,icmp,eco_i,SF,20,0,0,0,0,0,...,57,1.00,0.00,1.00,0.28,0.00,0.0,0.00,0.00,saint
4,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,86,0.31,0.17,0.03,0.02,0.00,0.0,0.83,0.71,mscan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22539,0,tcp,smtp,SF,794,333,0,0,0,0,...,141,0.72,0.06,0.01,0.01,0.01,0.0,0.00,0.00,normal
22540,0,tcp,http,SF,317,938,0,0,0,0,...,255,1.00,0.00,0.01,0.01,0.01,0.0,0.00,0.00,normal
22541,0,tcp,http,SF,54540,8314,0,0,0,2,...,255,1.00,0.00,0.00,0.00,0.00,0.0,0.07,0.07,back
22542,0,udp,domain_u,SF,42,42,0,0,0,0,...,252,0.99,0.01,0.00,0.00,0.00,0.0,0.00,0.00,normal


In [13]:
entire_df = pd.concat([train_df, test_df], axis=0)
entire_df = entire_df.reset_index()

In [14]:
entire_df

,index,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,0,tcp,ftp_data,SF,491,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,1,0,udp,other,SF,146,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,2,0,tcp,private,S0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
3,3,0,tcp,http,SF,232,8153,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,4,0,tcp,http,SF,199,420,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148512,22539,0,tcp,smtp,SF,794,333,0,0,0,...,141,0.72,0.06,0.01,0.01,0.01,0.00,0.00,0.00,normal
148513,22540,0,tcp,http,SF,317,938,0,0,0,...,255,1.00,0.00,0.01,0.01,0.01,0.00,0.00,0.00,normal
148514,22541,0,tcp,http,SF,54540,8314,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.07,0.07,back
148515,22542,0,udp,domain_u,SF,42,42,0,0,0,...,252,0.99,0.01,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [15]:
y_data = train_df.pop("label")

In [16]:
class_index_dict = {
    'normal': 0,
    'back': 1,
    'buffer_overflow': 2,
    'ftp_write': 3,
    'guess_passwd': 4,
    'imap': 5,
    'ipsweep': 6,
    'land': 7,
    'loadmodule': 8,
    'multihop': 9,
    'neptune': 10,
    'nmap': 11,
    'perl': 12,
    'phf': 13,
    'pod': 14,
    'portsweep': 15,
    'rootkit': 16,
    'satan': 17,
    'smurf': 18,
    'spy': 19,
    'teardrop': 20,
    'warezclient': 21,
    'warezmaster': 22
}

In [17]:
y_data = y_data.replace(class_index_dict)

In [18]:
y_data

0          0
1          0
2         10
3          0
4          0
          ..
125968    10
125969     0
125970     0
125971    10
125972     0
Name: label, Length: 125973, dtype: int64

In [19]:
category_class_list = ["protocol_type", "service", "flag"]
numerical_class_list = [column for column in train_df.columns if column not in category_class_list]

numerical_x_data = train_df[numerical_class_list]
categorical_x_data = train_df[category_class_list]

In [20]:
numerical_x_data

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,491,0,0,0,0,0,0,0,0,...,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00
1,0,146,0,0,0,0,0,0,0,0,...,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00
2,0,0,0,0,0,0,0,0,0,0,...,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00
3,0,232,8153,0,0,0,0,0,1,0,...,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01
4,0,199,420,0,0,0,0,0,1,0,...,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,0,0,0,0,0,0,0,0,0,...,255,25,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00
125969,8,105,145,0,0,0,0,0,0,0,...,255,244,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00
125970,0,2231,384,0,0,0,0,0,1,0,...,255,30,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00
125971,0,0,0,0,0,0,0,0,0,0,...,255,8,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00


In [21]:
categorical_x_data

,protocol_type,service,flag
0,tcp,ftp_data,SF
1,udp,other,SF
2,tcp,private,S0
3,tcp,http,SF
4,tcp,http,SF
...,...,...,...
125968,tcp,private,S0
125969,udp,private,SF
125970,tcp,smtp,SF
125971,tcp,klogin,S0


In [27]:
categorical_x_data = pd.get_dummies(data=categorical_x_data)

In [30]:
categorical_x_data

,protocol_type_icmp,protocol_type_tcp,protocol_type_udp,service_IRC,service_X11,service_Z39_50,service_aol,service_auth,service_bgp,service_courier,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
125969,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
125970,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
125971,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [31]:
numerical_x_data

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,491,0,0,0,0,0,0,0,0,...,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00
1,0,146,0,0,0,0,0,0,0,0,...,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00
2,0,0,0,0,0,0,0,0,0,0,...,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00
3,0,232,8153,0,0,0,0,0,1,0,...,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01
4,0,199,420,0,0,0,0,0,1,0,...,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,0,0,0,0,0,0,0,0,0,...,255,25,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00
125969,8,105,145,0,0,0,0,0,0,0,...,255,244,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00
125970,0,2231,384,0,0,0,0,0,1,0,...,255,30,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00
125971,0,0,0,0,0,0,0,0,0,0,...,255,8,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00


In [33]:
concat_df = pd.concat([numerical_x_data, categorical_x_data], axis=1)

In [34]:
concat_df

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0,491,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,146,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,232,8153,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,0,199,420,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
125969,8,105,145,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
125970,0,2231,384,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
125971,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [35]:
concat_df.describe()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
count,125973.00000,1.259730e+05,1.259730e+05,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,...,125973.00000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000
mean,287.14465,4.556674e+04,1.977911e+04,0.000198,0.022687,0.000111,0.204409,0.001222,0.395736,0.279250,...,0.08917,0.012399,0.000818,0.019218,0.276655,0.002897,0.001008,0.000389,0.594929,0.002151
std,2604.51531,5.870331e+06,4.021269e+06,0.014086,0.253530,0.014366,2.149968,0.045239,0.489010,23.942042,...,0.28499,0.110661,0.028583,0.137292,0.447346,0.053750,0.031736,0.019719,0.490908,0.046332
min,0.00000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.00000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.00000,4.400000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,0.00000,2.760000e+02,5.160000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,42908.00000,1.379964e+09,1.309937e+09,1.000000,3.000000,3.000000,77.000000,5.000000,1.000000,7479.000000,...,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [36]:
normal_y_data = y_data[y_data == 0]
abnormal_y_data = y_data[y_data > 0]

In [42]:
normal_y_data_index_list = normal_y_data.index.to_list()

In [43]:
normal_y_data_index_list[:10]

[0, 1, 3, 4, 12, 16, 18, 19, 22, 23]

In [44]:
np.random.shuffle(normal_y_data_index_list)

In [45]:
normal_y_data_index_list[:10]

[110188, 109854, 116865, 58105, 59293, 44116, 89877, 20129, 124027, 48655]

In [46]:
normal_numerical_x_data = numerical_x_data.iloc[normal_y_data_index_list]
normal_numerical_y_data = y_data.iloc[normal_y_data_index_list]

In [15]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch
import tqdm.notebook as tqdm

In [16]:
device = "cuda:7"

In [17]:
normal_numerical_x_train, normal_numerical_x_test, normal_numerical_y_train, normal_numerical_y_test = train_test_split(
    normal_numerical_x_data, normal_numerical_y_data,
    test_size=0.2,
    random_state=777
)

NameError: name 'normal_numerical_x_data' is not defined

In [18]:
from dataset.nsl_kdd_dataset import NSLKDDDataFrameFactory

In [19]:
numerical_x_train, categorical_x_train, numerical_x_test, categorical_x_test = NSLKDDDataFrameFactory.preprocess(
    dataframe=NSLKDDDataFrameFactory.create("entire"),
    abnormal_sample_ratio=0.1,
    normal_abnormal_ratio=1.0,
    val_test_ratio=0.3,
    is_shuffle=True
)

[2020-12-11 12:21:39,121][INFO][nsl_kdd_dataset.py:98] Success to load dataset dataframe : train
[2020-12-11 12:21:39,188][INFO][nsl_kdd_dataset.py:98] Success to load dataset dataframe : test
[2020-12-11 12:21:40,148][INFO][nsl_kdd_dataset.py:110] Success to load dataset dataframe : entire
[2020-12-11 12:21:40,512][INFO][nsl_kdd_dataset.py:156] Entire data size : 148517
[2020-12-11 12:21:40,513][INFO][nsl_kdd_dataset.py:157] Normal data size : 77054
[2020-12-11 12:21:40,514][INFO][nsl_kdd_dataset.py:158] Abnormal data size : 7146
[2020-12-11 12:21:40,514][INFO][nsl_kdd_dataset.py:159] Normal test data size : 7146
[2020-12-11 12:21:40,515][INFO][nsl_kdd_dataset.py:160] Training data size : 69908
[2020-12-11 12:21:40,515][INFO][nsl_kdd_dataset.py:161] Test data size : 14292


In [20]:
numerical_x_train

,index,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
105537,105537,0,290,1896,0,0,0,0,0,1,...,255,255,1.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00
14517,14517,0,295,1824,0,0,0,0,0,1,...,156,255,1.00,0.00,0.01,0.02,0.01,0.0,0.0,0.00
61908,61908,0,46,132,0,0,0,0,0,0,...,255,238,0.93,0.01,0.00,0.00,0.00,0.0,0.0,0.00
132592,6619,0,250,2137,0,0,0,0,0,1,...,255,255,1.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00
80657,80657,0,223,321,0,0,0,0,0,1,...,3,255,1.00,0.00,0.33,0.07,0.00,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140819,14846,0,314,15256,0,0,0,0,0,1,...,255,255,1.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00
88050,88050,0,335,2140,0,0,0,0,0,1,...,255,255,1.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00
58224,58224,0,193,16060,0,0,0,0,0,1,...,3,255,1.00,0.00,0.33,0.05,0.33,0.0,0.0,0.01
2077,2077,0,343,10925,0,0,0,0,0,1,...,5,255,1.00,0.00,0.20,0.03,0.00,0.0,0.0,0.00


In [21]:
categorical_x_train

,protocol_type_icmp,protocol_type_tcp,protocol_type_udp,service_IRC,service_X11,service_auth,service_domain,service_domain_u,service_eco_i,service_ecr_i,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
105537,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
14517,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
61908,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
132592,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
80657,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140819,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
88050,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
58224,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2077,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [25]:
categorical_x_test

,protocol_type_icmp,protocol_type_tcp,protocol_type_udp,service_IRC,service_X11,service_Z39_50,service_auth,service_bgp,service_courier,service_csnet_ns,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
117752,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
115622,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
78670,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
124010,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
12178,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78285,0,1,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
114954,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
116732,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
73469,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [26]:
categorical_x_test.columns

Index(['protocol_type_icmp', 'protocol_type_tcp', 'protocol_type_udp',
       'service_IRC', 'service_X11', 'service_Z39_50', 'service_auth',
       'service_bgp', 'service_courier', 'service_csnet_ns', 'service_ctf',
       'service_daytime', 'service_discard', 'service_domain',
       'service_domain_u', 'service_echo', 'service_eco_i', 'service_ecr_i',
       'service_efs', 'service_exec', 'service_finger', 'service_ftp',
       'service_ftp_data', 'service_gopher', 'service_hostnames',
       'service_http', 'service_http_443', 'service_http_8001',
       'service_imap4', 'service_iso_tsap', 'service_klogin', 'service_kshell',
       'service_ldap', 'service_link', 'service_login', 'service_mtp',
       'service_name', 'service_netbios_dgm', 'service_netbios_ns',
       'service_netbios_ssn', 'service_netstat', 'service_nnsp',
       'service_nntp', 'service_ntp_u', 'service_other', 'service_pm_dump',
       'service_pop_2', 'service_pop_3', 'service_printer', 'service_private',
  

In [22]:
x_train = pd.concat([numerical_x_train, categorical_x_train], axis=1)
x_test = pd.concat([numerical_x_test, categorical_x_test], axis=1)

In [23]:
x_train

,index,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
105537,105537,0,290,1896,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
14517,14517,0,295,1824,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
61908,61908,0,46,132,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
132592,6619,0,250,2137,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
80657,80657,0,223,321,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140819,14846,0,314,15256,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
88050,88050,0,335,2140,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
58224,58224,0,193,16060,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
2077,2077,0,343,10925,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0


In [24]:
x_test

,index,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
117752,117752,0,339,339,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
115622,115622,0,234,700,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
78670,78670,0,283,7203,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
124010,124010,0,180,4199,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
12178,12178,0,219,1228,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78285,78285,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
114954,114954,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
116732,116732,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
73469,73469,9695,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [12]:
x_train = x_train.set_index("index")
x_test = x_test.set_index("index")

In [13]:
x_train

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
index,,,,,,,,,,,,,,,,,,,,,
121730,0,318,501,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
115367,0,292,1091,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
72379,0,319,5036,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
512,0,356,1125,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
6921,0,250,608,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32624,17044,146,105,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
21084,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
102734,7,122,11930,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [14]:
x_test

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
index,,,,,,,,,,,,,,,,,,,,,
56585,0,1156,386,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
32773,0,1158,327,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
9689,0,45,129,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1919,0,202,2807,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
29863,0,867,330,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18899,0,0,44,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
98607,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
12458,1,0,36,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0


In [50]:
scaler = preprocessing.StandardScaler()
scaler.fit(normal_numerical_x_train)

# normal_numerical_scaled_x_train = scaler.transform(normal_numerical_x_train)
# normal_numerical_scaled_x_test = scaler.transform(normal_numerical_x_test)

normal_numerical_x_train = scaler.transform(normal_numerical_x_train)
normal_numerical_x_test = scaler.transform(normal_numerical_x_test)

In [51]:
normal_numerical_x_train = normal_numerical_x_train.astype(np.float32)

In [52]:
normal_numerical_x_train

array([[-0.1296171 , -0.02922032, -0.04857096, ..., -0.10900601,
        -0.23886849, -0.23327689],
       [-0.1296171 , -0.02946403, -0.03588925, ..., -0.10900601,
         3.1329904 ,  4.222955  ],
       [-0.1296171 , -0.02988277, -0.06289963, ..., -0.10900601,
        -0.23886849, -0.23327689],
       ...,
       [-0.1296171 , -0.02905194,  0.03040895, ..., -0.10900601,
        -0.23886849, -0.23327689],
       [-0.1296171 , -0.0294308 , -0.01676938, ..., -0.10900601,
        -0.23886849, -0.23327689],
       [-0.1296171 , -0.02950613, -0.05659622, ..., -0.10900601,
         4.6145644 ,  4.844755  ]], dtype=float32)

In [53]:
normal_numerical_x_train[0]

array([-0.1296171 , -0.02922032, -0.04857096, -0.0096342 ,  0.        ,
       -0.0077871 , -0.09904359, -0.02899666,  0.6402983 , -0.02504833,
       -0.04461018, -0.0332574 , -0.02641163, -0.03439498, -0.02415309,
       -0.05685904,  0.        , -0.00430838, -0.1134    , -0.10406838,
       -0.17962001, -0.14369206, -0.14170437, -0.21778083, -0.22098188,
        0.21329077, -0.19913876, -0.46627972,  1.0558814 ,  0.6996068 ,
        0.58137345, -0.31338215, -0.47887042, -0.3771208 , -0.15231219,
       -0.10900601, -0.23886849, -0.23327689], dtype=float32)

In [54]:
len(normal_numerical_x_train), len(normal_numerical_x_test), len(normal_numerical_y_train), len(normal_numerical_y_test)

(53874, 13469, 53874, 13469)

In [44]:
class NSLKDDDataset(Dataset):
    
    def __init__(self, x, y):
        super(NSLKDDDataset, self).__init__()
        self.x = x
        self.y = y
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return torch.tensor(self.x[idx]), torch.tensor(self.y.iloc[idx])
#         return torch.tensor(self.x.iloc[idx]), torch.tensor(self.y.iloc[idx])

In [55]:
train_dataset = NSLKDDDataset(x=normal_numerical_x_train, y=normal_numerical_y_train)
test_dataset = NSLKDDDataset(x=normal_numerical_x_test, y=normal_numerical_y_test)

In [56]:
print(len(train_dataset[0][0]))
train_dataset[0]

38


(tensor([-0.1296, -0.0292, -0.0486, -0.0096,  0.0000, -0.0078, -0.0990, -0.0290,
          0.6403, -0.0250, -0.0446, -0.0333, -0.0264, -0.0344, -0.0242, -0.0569,
          0.0000, -0.0043, -0.1134, -0.1041, -0.1796, -0.1437, -0.1417, -0.2178,
         -0.2210,  0.2133, -0.1991, -0.4663,  1.0559,  0.6996,  0.5814, -0.3134,
         -0.4789, -0.3771, -0.1523, -0.1090, -0.2389, -0.2333]), tensor(0))

In [69]:
type(train_dataset[0][0])

torch.Tensor

In [57]:
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=32,
    shuffle=True,
    pin_memory=True
)

test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=32,
    shuffle=False,
    pin_memory=True
)